## <center>循环神经网络</center>

### 1 循环神经网络介绍
举个例子,很多课文我们可以顺序背诵,而不能倒序背诵.这种现象可以理解为我们大脑并不是简单的存储,而是链式的,有序的存储.这种存储方式很节省存储空间,对于中间状态的序列,没有直接记住,而是记住了计算方法.
<center><img src='./img/7/1.png' width='600'/></center>

### 2 RNN实现退位减法器

#### (1) 建立基本函数

In [79]:
import numpy as np
import copy
np.random.seed(0)

In [80]:
# 定义 sigmoid 函数
def sigmoid(x):
    output = 1./(1+np.exp(-x))
    return output
# 定义 sigmoid 函数的导数
def sigmoid_derivative(output):
    return output*(1.-output)

#### (2)建立二进制映射

In [81]:
# 构建一个这样的字典:{0:00000000,1:00000001,2:00000010,...,255:11111111}
int2binary = {}
binary_dim = 8
largest_number = np.power(2,binary_dim)
for i in range(largest_number):
    int2binary[i] = bin(i)[2:].zfill(binary_dim)

#### (3)定义参数

定义学习参数:隐含层的权重为 synapse_0,循环节点的权重为 synapse_h(输入节点16,输出节点16),输出层的权重为synapse_1(输入16节点,输出1节点).

In [82]:
learning_rate = 0.9 # 学习率
input_dim = 2       # 输入的维度是2,减数和被减数
hidden_dim = 16 
output_dim =1       # 输出维度

# 初始化网络
# 输入维度是2,隐藏维度是16,为了保证初始化参数的范围 [-0.05,0.05]
synapse_0 = (2*np.random.random((input_dim,hidden_dim))-1)*0.05
synapse_1 = (2*np.random.random((hidden_dim,output_dim))-1)*0.05
synapse_h = (2*np.random.random((hidden_dim,hidden_dim))-1)*0.05

# 用于存放反向传播的权重更新值
synapse_0_update = np.zeros_like(synapse_0)
synapse_1_update = np.zeros_like(synapse_1)
synapse_h_update = np.zeros_like(synapse_h)

#### (4) 准备样本数据

大致是这样的过程:
+ 建立循环生成样本数据,先生成两个数 a 和 b .如果 a<b 就交换位置.
+ 计算出相减的结果 c
+ 将三个数转换成二进制.

In [87]:
for j in range(10000):
    a_int = np.random.randint(largest_number)
    b_int = np.random.randint(largest_number)
    if a_int < b_int:
        t = b_int
        b_int = a_int
        a_int = t
    # 将三个数都转换为对应的二进制
    a = int2binary[a_int]
    b = int2binary[b_int]
    c = int2binary[a_int-b_int]
    # 模型初始化输出值为0,初始化总误差为0,定义 layer_deltas 存储反向传播过程中的循环层的误差,
    # layer_values 为隐含层的输出值,由于第一个数据传入是传入时,没有前面的隐含层输出值作为本次
    # 样本的输入,所以需要定义一个初始值,这里定义为0.1
    d = np.zeros_like(np.arange(8))
    overallError = 0
    layer_2_deltas = list()
    layer_1_values = list()
    # 一开始没有隐含层,所以初始化一下原始值为0.1
    layer_1_values.append(np.ones(hidden_dim)*0.1)

#### (5) 正向传播

循环遍历每个二进制位,从个位开始依次相减,并将中间隐藏层的输出传入下一个时间步,把每一个时间步的误差导数记录下来,同时统计总误差.

In [84]:
    for position in range(binary_dim):
        # 生成输入,从右到左,每次取两个输入数字的一个bit位
        x = np.array([[int(a[binary_dim-position-1]),int(b[binary_dim-position-1])]])
        y = np.array([[int(c[binary_dim-position-1])]])
        # 隐含层的输入,包含上一步的隐藏层和当层的输入,输入层需要经过sigmoid只有才能相加
        # (输入层+之前的隐藏层)->新的隐藏层
        layer_1 = sigmoid(np.dot(x,synapse_0))+np.dot(layer_1_values[-1],synapse_h)
        layer_2 = sigmoid(np.dot(layer_1,synapse_1))
        layer_2_error = y - layer_2
        layer_2_deltas.append(layer_2_error*sigmoid_derivative(layer_2))
        overallError += np.abs(layer_2_error[0]) #总误差
        # 记录每次预测的bit位
        d[binary_dim-position-1] = np.round(layer_2[0][0])
        # 将当前的隐藏层保存下来,下个时间序列用
        layer_1_values.append(copy.deepcopy(layer_1))

#### (6) 反向传播

反向传播是从最后一次往前反向计算误差,对于每一个当前的计算都需要下一次结果参与,反向计算从最后一次开始,它没有后一次的输入,所以需要初始化一个值作为其后一次的输入,这里初始化为0

In [85]:
    future_layer_1_delta = np.zeros(hidden_dim)
    # position=[0,1,2,3,4,5,6,7]
    for position in range(binary_dim):
        # 最后一次的两个输入
        x = np.array([[int(a[position]),int(b[position])]])
        # 当前时间点的隐藏层
        layer_1 = layer_1_values[-position-1]
        # 上一个时间点的隐藏层
        pre_layer_1 = layer_1_values[-position-2]
        # 当前时间点的输出层误差
        layer_2_delta = layer_2_deltas[-position-1]
        # 通过后一个时间点的隐藏层误差和当前时间点的输出层误差,计算当前时间点的隐藏层误差
        layer_1_delta = (np.dot(future_layer_1_delta,synapse_h.T)+
                         np.dot(layer_2_delta,synapse_1.T))*sigmoid_derivative(layer_1)
        # 等完成所有反向传播误差计算,才会更新权重矩阵,先暂时把更新矩阵存起来
        synapse_1_update += np.atleast_2d(layer_1).T.dot(layer_2_delta)
        synapse_h_update += np.atleast_2d(pre_layer_1).T.dot(layer_1_delta)
        synapse_0_update += x.T.dot(layer_1_delta)
        future_layer_1_delta = layer_1_delta

    # 完成所有反向传播之后,更新权重矩阵,并把矩阵变量清零
    synapse_0 += synapse_0_update*learning_rate
    synapse_1 += synapse_1_update*learning_rate
    synapse_h += synapse_h_update*learning_rate
    synapse_0_update *= 0
    synapse_1_update *= 0
    synapse_h_update *= 0

#### (7) 代码汇总